In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.activations import relu,softmax
from tensorflow.keras.applications import vgg16
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.layers import Flatten,Dense,BatchNormalization
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [2]:
datagen = ImageDataGenerator()

In [3]:
import os

In [4]:
batch_size = 20

In [5]:
conv_base = vgg16.VGG16(include_top=False,input_shape=(32,32,3))

58892288/58889256 [==============================] - 0s 0us/step


In [6]:
for l in conv_base.layers[::-1]:

    l.trainable = False

In [7]:
def freeze_layers(layer):

  trainable = True

  for l in conv_base.layers[::-1]:

    if l.name == layer:
      l.trainable = False
      trainable = False

    l.trainable = trainable

    return conv_base

In [8]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [9]:
base_dir = "/content/drive/MyDrive/DevanagariHandwrittenCharacterDataset"

In [10]:
train_dir = os.path.join(base_dir,"Train")
cv_dir = os.path.join(base_dir,"cv")
test_dir = os.path.join(base_dir,"Test")

In [11]:
import numpy as np

In [12]:
def extract_features(data_directory,sample_count):
  features = np.zeros(shape=(sample_count,1,1,512))
  labels = np.zeros(shape=(sample_count,46))
  generator = datagen.flow_from_directory(directory=data_directory,
                                           target_size=(32,32),color_mode="rgb",
                                           classes=os.listdir(data_directory),
                                           batch_size=20)
  i = 0
  batch_size=20
  for images_batch,labels_batch in generator:
    features_batch = conv_base.predict(images_batch)
    features[i*batch_size:(i+1)*batch_size] = features_batch
    labels[i*batch_size:(i+1)*batch_size] = labels_batch
    i += 1
    if i*batch_size >= sample_count:
      break
  return features,labels

In [ ]:
train_features, train_labels = extract_features(train_dir,62560)

Found 62560 images belonging to 46 classes.
